### Extract the logatirhm of a SU(3) gauge link
Dana's method

In [1]:
import numpy as np
import math

Gell-Mann matrices

In [2]:
def complex_tuple(*t):
    return tuple(map(np.complex128, t))

Nc = 3

# Gell-Mann matrices
id0 = complex_tuple(1, 0, 0, 0, 1, 0, 0, 0, 1)
s1 = complex_tuple(0, 1, 0, 1, 0, 0, 0, 0, 0)
s2 = complex_tuple(0, -1j, 0, 1j, 0, 0, 0, 0, 0)
s3 = complex_tuple(1, 0, 0, 0, -1, 0, 0, 0, 0)
s4 = complex_tuple(0, 0, 1, 0, 0, 0, 1, 0, 0)
s5 = complex_tuple(0, 0, -1j, 0, 0, 0, 1j, 0, 0)
s6 = complex_tuple(0, 0, 0, 0, 0, 1, 0, 1, 0)
s7 = complex_tuple(0, 0, 0, 0, 0, -1j, 0, 1j, 0)
s8 = complex_tuple(1 / math.sqrt(3), 0, 0, 0, 1 / math.sqrt(3), 0, 0, 0, -2 / math.sqrt(3))

slist = (id0, s1, s2, s3, s4, s5, s6, s7, s8)

SU(3) matrix operations

In [3]:
"""
    SU(3) group & algebra functions
"""

# SU(3) group elements are given by 3x3 complex matrices:
#   a[0] a[1] a[2]
#   a[3] a[4] a[5]
#   a[6] a[7] a[8]
# (layout corresponds to C-order of numpy array)

# su3 multiplication

# Generate the matrix multiplication code:
# print("\n".join(["r{} = ".format(j) + " + ".join(["a[{}] * b[{}]".format(3 * (j // 3) + i, 3 * i + (j % 3)) for i in range(3)]) for j in range(9)]))

def mul(a, b):
    """SU(3) multiplication: 3x3 matrix multiplication

    >>> a=[1,2,3,4,5,6,7,8,9]
    >>> b=[3,6,8,4,3,2,1,3,4]
    >>> c=mul(a,b)
    >>> c
    (14, 21, 24, 38, 57, 66, 62, 93, 108)

    # Check with numpy
    >>> import numpy as np
    >>> ma = np.asarray(a).reshape(3,3)
    >>> mb = np.asarray(b).reshape(3,3)
    >>> mc = np.matmul(ma, mb)
    >>> mc
    array([[ 14,  21,  24],
           [ 38,  57,  66],
           [ 62,  93, 108]])

    >>> tuple(mc.flatten()) == c
    True
    """
    r0 = a[0] * b[0] + a[1] * b[3] + a[2] * b[6]
    r1 = a[0] * b[1] + a[1] * b[4] + a[2] * b[7]
    r2 = a[0] * b[2] + a[1] * b[5] + a[2] * b[8]
    r3 = a[3] * b[0] + a[4] * b[3] + a[5] * b[6]
    r4 = a[3] * b[1] + a[4] * b[4] + a[5] * b[7]
    r5 = a[3] * b[2] + a[4] * b[5] + a[5] * b[8]
    r6 = a[6] * b[0] + a[7] * b[3] + a[8] * b[6]
    r7 = a[6] * b[1] + a[7] * b[4] + a[8] * b[7]
    r8 = a[6] * b[2] + a[7] * b[5] + a[8] * b[8]
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

EXP_MIN_TERMS = -1 # minimum number of terms in Taylor series
EXP_MAX_TERMS = 100 # maximum number of terms in Taylor series
EXP_ACCURACY_SQUARED = 1.e-40 # 1.e-32 # accuracy

# exponential map
def mexp(a):
    """ Calculate exponential using Taylor series

    mexp(a) = 1 + a + a^2 / 2 + a^3 / 6 + ...

    >>> a = id0
    >>> mexp(a)
    ((2.7182818284590455+0j), 0j, 0j, 0j, (2.7182818284590455+0j), 0j, 0j, 0j, (2.7182818284590455+0j))

    """
    res = id0
    t = id0
    for i in range(1, EXP_MAX_TERMS):
        t = mul(t, a)
        t = mul_s(t, 1/i)
        res = add(res, t)
        n = sq(t)  # TODO: Is it possible to improve performance by checking this not so often?
        if (i > EXP_MIN_TERMS) and (math.fabs(n.real) < EXP_ACCURACY_SQUARED):
            break
    else:
        # print("Exponential did not reach desired accuracy: {}".format(a))   # TODO: remove debugging code
        print("Exponential did not reach desired accuracy")  # TODO: remove debugging code
    return res

LOG_MIN_TERMS = -1 # minimum number of terms in Taylor series
LOG_MAX_TERMS = 100 # maximum number of terms in Taylor series
LOG_ACCURACY_SQUARED = 1.e-32 # 1.e-32 # accuracy

# logarithm map
def mlog(a):
    """
    Computes logarithm of a matrix using Taylor series

    mlog(a) = (a-id0) - (a-id0)^2 / 2 + (a-id0)^3 / 3 - (a-id0)^4 / 4 + ...

    Works for matrices close to identity (for example gauge links)
    """
    res = add(a, mul_s(id0, -1))
    t = add(a, mul_s(id0, -1))
    sign = -1
    for i in range(1, LOG_MAX_TERMS):
        t = mul(t, add(a, mul_s(id0, -1)))
        buff = mul_s(t, sign/(i+1))
        res = add(res, buff)
        sign = sign * (-1)
        n = sq(t) 
        if (i > LOG_MIN_TERMS) and (math.fabs(n.real) < LOG_ACCURACY_SQUARED):
            break
        # else:
            # print("Logarithm did not reach desired accuracy") 

    return res

# determinant
def det(a):
    res  = a[0] * (a[4] * a[8] - a[5] * a[7])
    res += a[1] * (a[5] * a[6] - a[3] * a[8])
    res += a[2] * (a[3] * a[7] - a[4] * a[6])
    return res

# group add: g0 = g0 + f * g1
def add(g0, g1):
    # Unfortunately, tuple creation from list comprehension does not work in numba:
    # see https://github.com/numba/numba/issues/2771
    #
    # result = tuple(g0[i] + g1[i] for i in range(9))
    # return result
    r0 = g0[0] + g1[0]
    r1 = g0[1] + g1[1]
    r2 = g0[2] + g1[2]
    r3 = g0[3] + g1[3]
    r4 = g0[4] + g1[4]
    r5 = g0[5] + g1[5]
    r6 = g0[6] + g1[6]
    r7 = g0[7] + g1[7]
    r8 = g0[8] + g1[8]
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

# multiply by scalar
def mul_s(g0, f):
    # Unfortunately, tuple creation from list comprehension does not work in numba:
    # see https://github.com/numba/numba/issues/2771
    #
    # result = tuple(f * g0[i] for i in range(9))
    # return result
    r0 = np.complex128(f) * g0[0]
    r1 = np.complex128(f) * g0[1]
    r2 = np.complex128(f) * g0[2]
    r3 = np.complex128(f) * g0[3]
    r4 = np.complex128(f) * g0[4]
    r5 = np.complex128(f) * g0[5]
    r6 = np.complex128(f) * g0[6]
    r7 = np.complex128(f) * g0[7]
    r8 = np.complex128(f) * g0[8]
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

# conjugate transpose
def dagger(a):
    r0 = a[0].conjugate()
    r1 = a[3].conjugate()
    r2 = a[6].conjugate()
    r3 = a[1].conjugate()
    r4 = a[4].conjugate()
    r5 = a[7].conjugate()
    r6 = a[2].conjugate()
    r7 = a[5].conjugate()
    r8 = a[8].conjugate()
    return r0, r1, r2, r3, r4, r5, r6, r7, r8

# trace
def tr(a):
    return a[0] + a[4] + a[8]

# trace ( a . dagger(a) )
def sq(a): # TODO: rename to tr_sq? or tr_abs_sq?
    """
    >>> a = (1,2,3,4,6,8,9,5,4)
    >>> ta = sq(a)
    >>> ta
    252

    >>> tma = tr(mul(a, dagger(a))).real
    >>> tma
    252

    >>> tma == ta
    True

    :param a:
    :return:
    """
    # return tr(mul(a, dagger(a))).real

    s = np.float64(0)
    for i in range(9):
        s += a[i].real * a[i].real + a[i].imag * a[i].imag
    return s

def check_unitary(u):  # TODO: remove debugging code
    x = mul(u, dagger(u))
    d = add(x, mul_s(id0, -1))
    s = sq(d)
    # if s > 1e-8:
    #    print("Unitarity violated")  # TODO: remove debugging code
    return s

Input Pooja's gauge link

In [4]:
# Pooja's gauge link
ux = complex_tuple(0.999773332297265+1.200901570480802E-002j, 9.337536969798815E-003+9.373516471449303E-003j,-1.157649128313104E-002+-3.300514581480528E-006j,
    -9.049799378893927E-003+9.456078543778054E-003j, 0.999470695182973+-2.638751157253443E-002j, 1.305022491015412E-002+-4.517083466599165E-003j,
    1.173464879673716E-002+2.220279658133394E-004j, -1.299671643839491E-002+-4.249900891929569E-003j, 0.999734175041685+1.438233914040814E-002j)

Check the unitarity of the gauge link

In [5]:
check_unitary(ux)

1.4823476966401162e-31

Compute the determinant of the gauge link

In [6]:
print("The determinant is", det(ux))

The determinant is (1.0000000000000002-3.470743971154097e-18j)


Extract the logarithm as a Taylor series $\boxed{\mathrm{ln}\, U=\displaystyle\sum_n\dfrac{(-1)^n(U-\mathcal{I})^n}{n}}$

In [7]:
lnux_taylor = np.array(mlog(ux))
lnux_taylor

array([ 1.92188480e-16+0.01200978j,  9.19523417e-03+0.0094164j ,
       -1.16575554e-02+0.00010938j, -9.19523417e-03+0.0094164j ,
        1.08408557e-16-0.02639329j,  1.30256890e-02-0.00438424j,
        1.16575554e-02+0.00010938j, -1.30256890e-02-0.00438424j,
       -4.53993708e-17+0.01438351j])

Extract the logaritm as the anti-hermitian part of the gauge link $\boxed{\mathrm{ln}\, U=\dfrac{1}{2}\left(U-U^\dagger\right)}$

In [8]:
lnux_uudag = np.array(mul_s(add(ux, mul_s(dagger(ux), -1)), 0.5))
lnux_uudag

array([ 0.        +0.01200902j,  0.00919367+0.0094148j ,
       -0.01165557+0.00010936j, -0.00919367+0.0094148j ,
        0.        -0.02638751j,  0.01302347-0.00438349j,
        0.01165557+0.00010936j, -0.01302347-0.00438349j,
        0.        +0.01438234j])

Extract the logarithm using ```Scipy```

In [9]:
from scipy.linalg import logm
lnux_scipy = np.reshape(logm(np.reshape(ux, (Nc, Nc))), Nc*Nc)
lnux_scipy

array([-2.18141477e-16+0.01200978j,  9.19523417e-03+0.0094164j ,
       -1.16575554e-02+0.00010938j, -9.19523417e-03+0.0094164j ,
       -3.41740525e-16-0.02639329j,  1.30256890e-02-0.00438424j,
        1.16575554e-02+0.00010938j, -1.30256890e-02-0.00438424j,
       -1.73472348e-16+0.01438351j])

Compare ```Scipy``` with the Taylor series logarithm

In [10]:
lnux_taylor - lnux_scipy

array([ 4.10329958e-16-1.04083409e-17j,  2.02962647e-16+3.62557206e-16j,
        1.26634814e-16-5.69409428e-16j,  2.22044605e-16-5.39499001e-16j,
        4.50149081e-16+1.80411242e-16j,  1.21430643e-17+1.26634814e-16j,
       -9.02056208e-17+5.02663232e-16j, -4.51028104e-17-3.20923843e-16j,
        1.28072977e-16-4.85722573e-17j])

Compare ```Scipy``` with the anti-hermitian logarithm

In [11]:
lnux_uudag - lnux_scipy

array([ 2.18141477e-16-7.64224652e-07j, -1.56599257e-06-1.60364535e-06j,
        1.98535885e-06-1.86182703e-08j,  1.56599257e-06-1.60364535e-06j,
        3.41740525e-16+5.77598560e-06j, -2.21832781e-06+7.46662893e-07j,
       -1.98535885e-06-1.86182693e-08j,  2.21832781e-06+7.46662893e-07j,
        1.73472348e-16-1.16848827e-06j])

- - -
Load Pooja's Fortran subroutines for extracting the logarithm

In [12]:
%load_ext fortranmagic

Extract the logarithm as a Taylor series

In [13]:
%%fortran --extra "-DF2PY_REPORT_ON_ARRAY_COPY=1"
subroutine mlog_taylor_pooja(input, output)
   complex(8), dimension(3, 3), intent(in) :: input
   complex(8), dimension(3, 3), intent(out) :: output

   complex(8), parameter :: i = (0.0D0, 1.0D0)	
   complex(8), dimension(3, 3) :: II, mat_U, mat_U_I, mat_U_I_pow, log_U
   integer :: j5

   mat_U = input

   II = 0.0D0	
   II(1,1) = (1.0D0, 0.0D0) ;	II(2,2) = (1.0D0, 0.0D0) ;	II(3,3) = (1.0D0, 0.0D0)

   mat_U_I      =  mat_U - II		
   mat_U_I_pow  =  II				
   log_U = (0.0D0, 0.0D0)	

   DO j5 = 1, 100
      mat_U_I_pow  =  MATMUL(mat_U_I, mat_U_I_pow)
      log_U        =  log_U + (((-1.0D0)**(j5+1) )/DBLE(j5))*mat_U_I_pow				
   ENDDO

   output = log_U

end subroutine

In [14]:
ux_array = np.reshape(np.array(ux), (Nc, Nc))
lnux_taylor_pooja = mlog_taylor_pooja(ux_array)
lnux_taylor_pooja

array([[ 1.92188190e-16+0.01200978j,  9.19523417e-03+0.0094164j ,
        -1.16575554e-02+0.00010938j],
       [-9.19523417e-03+0.0094164j ,  1.08407143e-16-0.02639329j,
         1.30256890e-02-0.00438424j],
       [ 1.16575554e-02+0.00010938j, -1.30256890e-02-0.00438424j,
        -4.53995715e-17+0.01438351j]])

Compare our results

In [15]:
np.reshape(np.array(lnux_taylor), (Nc, Nc)) - lnux_taylor_pooja

array([[2.90822564e-22+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 1.41389243e-21+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 2.00668696e-22+0.j]])

Extract the logarithm from the anit-hermitian part

In [16]:
%%fortran --extra "-DF2PY_REPORT_ON_ARRAY_COPY=1"
subroutine mlog_uudag_pooja(input, output)
   complex(8), dimension(3, 3), intent(in) :: input
   complex(8), dimension(3, 3), intent(out) :: output

   complex(8), parameter :: i = (0.0D0, 1.0D0)	
   complex(8), dimension(3, 3) :: mat_U, mat_U_dag, log_U
   integer :: j5

   mat_U = input

   mat_U_dag  = DCONJG(TRANSPOSE(mat_U))				
   log_U =	(1/2.0D0)*(mat_U - mat_U_dag)

   output = log_U

end subroutine

In [17]:
lnux_uudag_pooja = mlog_uudag_pooja(ux_array)
lnux_uudag_pooja

array([[ 0.        +0.01200902j,  0.00919367+0.0094148j ,
        -0.01165557+0.00010936j],
       [-0.00919367+0.0094148j ,  0.        -0.02638751j,
         0.01302347-0.00438349j],
       [ 0.01165557+0.00010936j, -0.01302347-0.00438349j,
         0.        +0.01438234j]])

Compare our results

In [18]:
np.reshape(np.array(lnux_uudag), (Nc, Nc)) - lnux_uudag_pooja

array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])